In [1]:
### Import Packages

import pandas as pd
import numpy as np
import os
import pickle

In [2]:
### Read in all fielding data

teams = pd.read_csv('playoff_teams.csv')
all_fielding_stats = pd.read_csv('baseball_data/fielding.csv')

/var/folders/fh/fzyqqz2s1vzcm3v06kz1h29w0000gp/T/ipykernel_8324/2140893708.py:2: DtypeWarning: Columns (3,4,19) have mixed types. Specify dtype option on import or set low_memory=False.
  all_fielding_stats = pd.read_csv('baseball_data/fielding.csv')


In [3]:
### Select features

# Features used before feature engineering
fielding_features = ['id', 'd_pos', 'd_ifouts', 'd_po', 'd_a', 'd_e', 'd_dp', 'd_tp', 'd_pb', 'd_sb', 'd_cs']
# Final feature selection
fielding_features_final = ['d_ifouts', 'd_po', 'd_a', 'd_e', 'd_dp', 'd_tp', 'd_pb', 'd_sb', 'd_cs', 'n_pos']

In [ ]:
### Loop through all playoff teams and save data

# Define number of players in list
max_players = 25
num_features = len(fielding_features_final)

all_seasons = np.empty([0, max_players, num_features])

## Start loop
for index, team in teams.iterrows():
    print(f"Index: {index}, Year: {team['Year']} Team: {team['Team']}, Level of Success: {team['Level of Success']}")
    # Filter all stats to appropriate team
    temp_stats = all_fielding_stats[all_fielding_stats['team'] == team['Team']][all_fielding_stats['date'].astype(str).str.startswith(str(team['Year']))][all_fielding_stats['gametype'] == 'regular'][fielding_features]
    n_pos = temp_stats[['id', 'd_pos']].groupby(by='id').nunique()
    temp_stats = temp_stats.groupby(by='id').sum()

    ## Feature Engineering
    # Total number of hits / Total number of at-bats
    temp_stats['n_pos'] = n_pos

    # Sort players by total number of outs played and then by total number of errors
    # This will place the most-used players at the top of the list and the worst performing players on top in case of a tie.
    temp_stats = temp_stats.sort_values(by=['d_ifouts', 'd_e'], ascending=[False, False])
    temp_stats = temp_stats.drop(columns=['d_pos'])
    temp_stats = temp_stats.to_numpy()

    # Adding padding to teams which used fewer than max_players for fielding throughout the season
    padding = max_players - temp_stats.shape[0]
    if padding > 0:
        temp_stats = np.pad(temp_stats, ((0, padding), (0, 0)), mode='constant')
    elif padding < 0:
        temp_stats = temp_stats[:max_players,]
    temp_stats = np.reshape(temp_stats, (1, max_players, num_features))

    all_seasons = np.vstack((all_seasons, temp_stats))

# Save data to .pkl file
with open('all_fielding_data.pkl', 'wb') as file:
    pickle.dump(all_seasons, file)

print("Data saved to 'all_fielding_data.pkl'")